## Testing implementations of LibFM

By **LibFM** I mean is an approach to solve classification and regression problems.
This approach is frequently used in recommendation systems, because it generalizes the matrix decompositions. 

**LibFM** proved to be quite useful to deal with highly categorical data (user id / movie id / movie language / site id / advertisement id / etc.).

Implementations tested


* Original and widely known implementation was written by Steffen Randle (and available on [github](https://github.com/srendle/libfm)).
    * contains SGD, SGDA, ALS and MCMC optimizers
    * command-line interface, does not have official python / R wrapper
    * does not provide a way to save / load trained formula. Each time you want to predict something, you need to restart training process 
    * has some extensions (that almost nobody uses)
    * supports linux, mac os
    * has non-oficial [pythonic wrapper](https://github.com/jfloff/pywFM)


* FastFM ([github repo](https://github.com/ibayer/fastFM))
    * claimed to be faster in the author's article
    * has both command-line interface and convenient python wrapper, which *almost* follows scikit-learn conventions.
    * supports SGD, ALS and MCMC optimizers
    * supports save / load (for the except of MCMC)
    * supports linux, mac os (though some issues with mac os)
    
    
* pylibFM ([github repo](https://github.com/coreylynch/pyFM))
    * uses SGDA
    * pythonic library implemented with cython
    * save / load operates normally
    * supports any platform, provided cython operates normally
    * slow and requires additional tuning, the number of iterations is reduced for pylibFM in tests
    
None of the libraries are pip-installable and all libraries need some manual setup. FastFM is the only to install itself normally into site-packages.

## What is tested

ALS (alternating least squares) is very useful optimization technique for factorization models, however
there is still one parameter one has to pass - namely, regularization. Quality of classification / regression is quite sensible to this parameter, so for fast tests data analyst prefers to leave the question of selecting regularization to machine learning.

MCMC is ususally proposed as a solution: optimization algorithm should find itself the optimal regularization. 
MCMC uses however some priors (which don't influence the result that much).

So I am testing the quality libraries provide **without additional tuning** to check how bayesian inference / other heuristics work.

## Logistic regression

Logistic regression is used as a stable **baseline**, because it is basic method to work with highly categorical data.

However, logistic regression, for instance, does not encounter the relation between user variables and movie variables (in the context of movie recommendations), so this approach is not able to provide any senseful recommendations.



## Defining functions for benchmarking

In [12]:
import numpy
import pandas
import load_problems
import cPickle as pickle
from sklearn.metrics import roc_auc_score, mean_squared_error

In [2]:
from fastFM.mcmc import FMClassification, FMRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.datasets import dump_svmlight_file

In [3]:
LIBFM_PATH = '/moosefs/ipython_env/python_libfm/bin/libFM'
PYLIBFM_PATH = '/moosefs/ipython_env/python_pylibFM/'

import sys
if PYLIBFM_PATH not in sys.path:
    sys.path.insert(0, PYLIBFM_PATH)
import pylibfm


def fitpredict_logistic(trainX, trainY, testX, classification=True, **params):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    if classification:
        clf = LogisticRegression(**params)
        clf.fit(trainX, trainY)
        return clf.predict_proba(testX)[:, 1]
    else:
        clf = Ridge(**params)
        clf.fit(trainX, trainY)
        return clf.predict(testX)


def fitpredict_fastfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    if classification:
        clf = FMClassification(rank=rank, n_iter=n_iter)
        return clf.fit_predict_proba(trainX, trainY, testX)
    else:
        clf = FMRegression(rank=rank, n_iter=n_iter)
        return clf.fit_predict(trainX, trainY, testX)        

def fitpredict_libfm(trainX, trainY, testX, classification=True, rank=8, n_iter=100):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    train_file = 'libfm_train.txt'
    test_file = 'libfm_test.txt'
    with open(train_file, 'w') as f:
        dump_svmlight_file(trainX, trainY, f=f)
    with open(test_file, 'w') as f:
        dump_svmlight_file(testX, numpy.zeros(testX.shape[0]), f=f)
    task = 'c' if classification else 'r'
    console_output = !$LIBFM_PATH -task $task -method mcmc -train $train_file -test $test_file -iter $n_iter -dim '1,1,$rank' -out output.libfm
    
    libfm_pred = pandas.read_csv('output.libfm', header=None).values.flatten()
    return libfm_pred

def fitpredict_pylibfm(trainX, trainY, testX, classification=True, rank=8, n_iter=10):
    encoder = OneHotEncoder(handle_unknown='ignore').fit(trainX)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    task = 'classification' if classification else 'regression'
    fm = pylibfm.FM(num_factors=rank, num_iter=n_iter, verbose=False, task=task)
    if classification:
        fm.fit(trainX, trainY)
    else:
        fm.fit(trainX, trainY * 1.)
    return fm.predict(testX)



In [4]:
from collections import OrderedDict
import time

all_results = OrderedDict()
try:
    with open('./saved_results.pkl') as f:
        all_results = pickle.load(f)
except:
    pass

def test_on_dataset(trainX, testX, trainY, testY, task_name, classification=True, use_pylibfm=True):
    algorithms = OrderedDict()
    algorithms['logistic'] = fitpredict_logistic
    algorithms['libFM']    = fitpredict_libfm
    algorithms['fastFM']   = fitpredict_fastfm
    if use_pylibfm:
        algorithms['pylibfm']  = fitpredict_pylibfm
    
    results = pandas.DataFrame()
    for name, fit_predict in algorithms.items():
        start = time.time()
        predictions = fit_predict(trainX, trainY, testX, classification=classification)
        spent_time = time.time() - start
        results.ix[name, 'time'] = spent_time
        if classification:
            results.ix[name, 'ROC AUC'] = roc_auc_score(testY, predictions)
        else:
            results.ix[name, 'RMSE'] = numpy.mean((testY - predictions) ** 2) ** 0.5
            
    all_results[task_name] = results
    with open('saved_results.pkl', 'w') as f:
        pickle.dump(all_results, f)
        
    return results

## Testing on movielens-100k dataset, only ids

In [5]:
trainX, testX, trainY, testY = load_problems.load_problem_movielens_100k(all_features=False)
trainX.head()

,user,movie
98980,810,900
69824,803,754
9928,51,286
75599,734,180
95621,896,95


In [6]:
test_on_dataset(trainX, testX, trainY, testY, task_name='ml100k, ids', classification=False)

,time,MSE
logistic,0.054624,0.888817
libFM,8.531124,0.838112
fastFM,4.905940,0.837561
pylibfm,12.668193,0.895734


## Testing on movielens-100k dataset, with additional information

In [7]:
trainX, testX, trainY, testY = load_problems.load_problem_movielens_100k(all_features=True)
trainX.head()

,user,movie,age,gender,occupation,zip,released,unknown,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
98980,692,1310,33,0,7,615,68,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69824,931,528,48,1,3,59,57,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9928,216,553,12,1,13,110,67,0,1,1,...,0,0,0,0,0,0,0,0,0,0
75599,798,498,39,0,0,166,30,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95621,910,547,27,0,20,397,68,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
test_on_dataset(trainX, testX, trainY, testY, task_name='ml100k', classification=False)

,time,MSE
logistic,1.738463,0.888075
libFM,47.375382,0.802437
fastFM,52.881285,0.803790
pylibfm,56.293395,NaN


## Testing on movielens-1m dataset, only ids

In [11]:
trainX, testX, trainY, testY = load_problems.load_problem_movielens_1m(all_features=False)
trainX.head()

load_problems.py:73: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  names=['user', 'movie', 'rating', 'timestamp'], header=None)


,user,movie
610738,3703,3541
324752,1923,756
808217,4836,1288
133807,866,1106
431857,2630,2857


In [12]:
test_on_dataset(trainX, testX, trainY, testY, task_name='ml-1m,ids', classification=False)

,time,MSE
logistic,0.938954,0.829407
libFM,194.550568,0.741458
fastFM,285.614966,0.736687
pylibfm,138.188093,0.761485


## Testing on movielens-1m dataset, with additional information

In [15]:
trainX, testX, trainY, testY = load_problems.load_problem_movielens_1m(all_features=True)
trainX.head()

load_problems.py:78: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  names=['user', 'gender', 'age', 'occupation', 'zip'], header=None)
load_problems.py:79: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  movies = pandas.read_csv(folder + '/movies.dat', sep='::', names=['movie', 'title', 'genres'], header=None)


,user,movie,gender,age,occupation,zip,0,1,2,3,...,10,11,12,13,14,15,16,17,18,19
610738,5245,2240,0,1,0,2168,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
324752,4260,2213,0,3,6,346,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
808217,481,1205,1,2,14,1878,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
133807,973,1108,1,3,19,3140,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
431857,1344,2302,0,2,2,2336,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [16]:
test_on_dataset(trainX, testX, trainY, testY, task_name='ml-1m', classification=False)

,time,MSE
logistic,20.395793,0.829965
libFM,668.852091,0.722149
fastFM,1120.280621,0.727162
pylibfm,564.452579,NaN


## Test on flights dataset - 1m

In [17]:
trainX, testX, trainY, testY = load_problems.load_problem_flight(large=False, convert_to_ints=False)
trainX.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-4,c-26,c-2,1828,XE,LEX,IAH,828
1,c-12,c-11,c-1,1212,UA,DEN,MCI,533
2,c-10,c-1,c-6,935,OH,HSV,CVG,325
3,c-11,c-26,c-6,930,OH,JFK,PNS,1028
4,c-12,c-6,c-2,1350,MQ,DFW,LBB,282


In [24]:
trainX, testX, trainY, testY = load_problems.load_problem_flight(large=False, convert_to_ints=True)
trainX.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,7,19,2,8,21,157,133,6
1,4,3,1,4,18,79,172,4
2,2,1,6,2,15,129,72,2
3,3,19,6,2,15,147,220,7
4,4,28,2,5,13,80,155,2


In [25]:
len(trainX)

1000000

In [23]:
test_on_dataset(trainX, testX, trainY, testY, task_name='flight1m', classification=True)


,time,ROC AUC
logistic,26.408147,0.715099
libFM,441.534214,0.720484
fastFM,667.197644,0.718840
pylibfm,316.149734,0.711200


## Test on flights dataset - 10m

In [ ]:
trainX, testX, trainY, testY = load_problems.load_problem_flight(large=True, convert_to_ints=True)
trainX.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,8,3,4,0,15,37,29,1
1,6,16,3,8,1,149,157,9
2,3,13,7,0,13,29,37,1
3,11,27,6,4,14,80,198,7
4,7,19,2,1,12,216,155,9


In [11]:
test_on_dataset(trainX, testX, trainY, testY, task_name='flight10m', classification=True, use_pylibfm=False)

,time,ROC AUC
logistic,307.085924,0.715754
libFM,8500.038059,0.724258
fastFM,10718.261802,0.721615


In [56]:
logs = Logistic_My(regularization=5., n_iterations=20)
train_start = 1
logs.fit(trainX[train_start::2], trainY[train_start::2])

0 206240.709146
1 203811.154365
2 203519.873727
3 203442.513614
4 203396.616017
5 203360.715704
6 203331.242334
7 203306.829145
8 203286.503109
9 203269.486079
10 203255.157185
11 203243.024488
12 203232.69835
13 203223.86884
14 203216.287788
15 203209.754924
16 203204.10732
17 203199.211364
18 203194.956638
19 203191.251245


<__main__.Logistic_My instance at 0x7f8908cb8f38>

In [57]:
print roc_auc_score(trainY[train_start::2], logs.predict_proba(trainX[train_start::2]))
print roc_auc_score(trainY[1 - train_start::2], logs.predict_proba(trainX[1 - train_start::2]))

# logs.predict_train(trainX[train_start::2], trainY[train_start::2])

print roc_auc_score(trainY[train_start::2], logs.predict_train(trainX[train_start::2]))


0.803038889948
0.786838488762
0.781306427766
